In [ ]:
# ==================== app.py (código principal) ====================
import pandas as pd
import numpy as np
from dash import Dash
import dash_bootstrap_components as dbc

import Funcioness as F  # 👈 tu módulo

# -------------------- CARGA DE DATOS --------------------
EXCEL = "Base_Empresa_BigData_Limpio.xlsx"
df_envios   = pd.read_excel(EXCEL, sheet_name="Envíos")
df_ordenes  = pd.read_excel(EXCEL, sheet_name="Órdenes")
df_detalle  = pd.read_excel(EXCEL, sheet_name="Detalle_Orden")
df_producto = pd.read_excel(EXCEL, sheet_name="Productos")

# -------------------- UNIONES NECESARIAS --------------------
# df_envios ← agrega ID_Producto y Nombre_Producto_Real
df_envios = df_envios.merge(df_detalle[["ID_Orden", "ID_Producto"]], on="ID_Orden", how="left")
df_envios = df_envios.merge(
    df_producto[["ID_Producto", "Nombre_Producto_Real"]],
    on="ID_Producto",
    how="left"
)

# df_ordenes_full (Órdenes + Detalle + Producto)
df_ordenes_full = df_ordenes.merge(df_detalle, on="ID_Orden", how="inner")
df_ordenes_full = df_ordenes_full.merge(df_producto, on="ID_Producto", how="left")

# -------------------- COLUMNAS VIRTUALES/DERIVADAS --------------------
df_ordenes_full["Fecha"] = pd.to_datetime(df_ordenes_full["Fecha"], errors="coerce")
df_ordenes_full["Ingreso_Item"] = df_ordenes_full["Cantidad"] * df_ordenes_full["Precio"]
df_ordenes_full["Día del Año"] = df_ordenes_full["Fecha"].dt.dayofyear
df_ordenes_full["Fin_de_Semana"] = df_ordenes_full["Fecha"].dt.weekday >= 5
df_ordenes_full["Descuento"] = df_ordenes_full.get("Descuento", 0).fillna(0)
df_ordenes_full["Descuento_Aplicado"] = df_ordenes_full["Descuento"] > 0
df_ordenes_full["Producto_Premium"] = df_ordenes_full["Precio"] > df_ordenes_full["Precio"].median()

# Estandarización método de pago (si no existe)
if "Método_Pago_Estandarizado" not in df_ordenes_full.columns:
    if "Método_Pago" in df_ordenes_full.columns:
        df_ordenes_full["Método_Pago_Estandarizado"] = df_ordenes_full["Método_Pago"].fillna("Transferencia")
    else:
        df_ordenes_full["Método_Pago_Estandarizado"] = "Transferencia"

# -------------------- EVENTOS (feriados/campañas) --------------------
eventos_df = pd.DataFrame({
    "Fecha": pd.to_datetime([
        "2023-02-14", "2023-09-18", "2023-11-24", "2023-12-25",
        "2024-02-14", "2024-09-18", "2024-11-24", "2024-12-25"
    ]),
    "Evento": [
        "San Valentín", "Fiestas Patrias", "Black Friday", "Navidad",
        "San Valentín", "Fiestas Patrias", "Black Friday", "Navidad"
    ]
})

# -------------------- ENRIQUECER (usa tu Funcioness.generar_nuevas_features) --------------------
df_enriquecido = F.generar_nuevas_features(
    df=df_ordenes_full,
    eventos_df=eventos_df,
    meses_estacionales=None,
    detectar_meses_fn=F.detectar_meses_estacionales_por_outliers,
    anios=[2023, 2024]  # entrenamos en 23-24; incluimos 2025 para predicción
)

# Categorización de categoría (si existe)
if "Categoría" in df_enriquecido.columns:
    df_enriquecido["Categoría_categorizada"] = df_enriquecido["Categoría"].astype("category")
else:
    df_enriquecido["Categoría_categorizada"] = "General"

# -------------------- FEATURES PARA EL MODELO --------------------
columnas_enriquecidas = [
    "Método_Pago_Estandarizado",
    "Día del Año",
    "Categoría_categorizada",
    "Fin_de_Semana",
    "Descuento_Aplicado",
    "Producto_Premium",
    "Dia_Feriado",
    "Es_Estacional",
]

# -------------------- ENTRENAR (predecir CANTIDAD) --------------------
modelo_cant, mae_c, rmse_c, _ = F.modelo_random_forest_total_compra(
    df=df_enriquecido,
    features=columnas_enriquecidas,
    target="Cantidad",
    anios_entrenamiento=[2023, 2024],
    columna_fecha="Fecha"
)

# Predicción (incluye 2025; si hay filas 2025, quedará Prediccion_Cantidad)
df_enriquecido = F.predecir_datos_avanzado(
    df=df_enriquecido,
    modelo=modelo_cant,
    columnas_features=columnas_enriquecidas,
    anios_prediccion=[2023, 2024, 2025],
    columna_fecha="Fecha",
    nombre_columna_pred="Prediccion_Cantidad"
)

# -------------------- OPCIONES PARA LOS SEGMENTADORES --------------------
years_opts = sorted(df_enriquecido["Fecha"].dropna().dt.year.unique().astype(int).tolist())
centros_opts = sorted(df_envios["Centro_Distribucion"].dropna().unique().tolist())
if not centros_opts:
    centros_opts = ["Centro 1"]  # fallback amigable si faltan datos

# -------------------- COSTOS (Q* / ROP) --------------------
costos_orden = {
    "Smartwatch": 1800, "Jeans": 500, "Smartphone": 2200, "Zapatillas": 600,
    "Escritorio de Oficina": 2000, "Audífonos Bluetooth": 800, "Tablet": 1600,
    "Impresora Láser": 1900, "Cámara de Seguridad": 1700, "Silla Ergonómica": 1800,
    "Monitor LED 24": 1500, "Teclado Mecánico": 400, "Polera Deportiva": 450,
    "Juego de Sábanas": 750, "Lámpara LED": 600
}
costos_mantencion = {
    "Smartwatch": 100, "Jeans": 40, "Smartphone": 120, "Zapatillas": 50,
    "Escritorio de Oficina": 150, "Audífonos Bluetooth": 70, "Tablet": 110,
    "Impresora Láser": 130, "Cámara de Seguridad": 90, "Silla Ergonómica": 130,
    "Monitor LED 24\"": 100, "Teclado Mecánico": 30, "Polera Deportiva": 35,
    "Juego de Sábanas": 60, "Lámpara LED": 45
}

# -------------------- DASH --------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout con CSS inline para sliders dentro de Funcioness.crear_layout
app.layout = F.crear_layout(app, years_opts, centros_opts)

# Callbacks (Q*, ROP, LT). Usa Cantidad_Efectiva: real excepto 2025 -> Prediccion_Cantidad
F.registrar_callbacks(app, df_enriquecido, df_envios, costos_orden, costos_mantencion)

if __name__ == "__main__":
    app.run_server(debug=True)








AttributeError: module 'dash.html' has no attribute 'Style'